In [ ]:
pip install asyncio
pip install requests

In [ ]:
import pandas as pd
import numpy as np
import os  # For handling file paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report  # Import classification report

# Paths to the two datasets
file_path1 = "C:/Users/Ble Salva/Documents/EA/Dissert(2024)/feeds1.csv"  # First dataset
file_path2 = "C:/Users/Ble Salva/Documents/EA/Dissert(2024)/feeds (1).csv"  # Second dataset

# Load the datasets
pdata1 = pd.read_csv(file_path1)
pdata2 = pd.read_csv(file_path2)
print("Datasets Loaded")
print(pdata1.head(), pdata2.head())

# Select relevant columns and map the headers to the appropriate fields for both datasets
data1 = pdata1[['field1', 'field2', 'field3', 'field4', 'field5', 'field6', 'field7']]
data1.columns = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label']

data2 = pdata2[['field1', 'field2', 'field3', 'field4', 'field5', 'field6', 'field7']]
data2.columns = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label']

# Combine the two datasets into one
combined_data = pd.concat([data1, data2], ignore_index=True)
print(combined_data)
# Handle missing values by removing rows with NaN values
combined_data.dropna(inplace=True)

# Convert data to float for numeric columns
for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'label']:
    combined_data[col] = pd.to_numeric(combined_data[col], errors='coerce')

# Drop any rows that could not be converted to numeric values (if any exist)
combined_data.dropna(inplace=True)

# Normalize accelerometer and gyroscope data
scaler = StandardScaler()
combined_data[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']] = scaler.fit_transform(
    combined_data[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']]
)

# Encode the labels (gestures)
label_mapping = {0: 'left', 1: 'right', 2: 'up', 3: 'down', 4: 'rest'}  
combined_data['label'] = combined_data['label'].map(label_mapping) 
le = LabelEncoder()
combined_data['label'] = le.fit_transform(combined_data['label'])

# Split the data into features (X) and labels (y)
X = combined_data[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']]
y = combined_data['label']

# Split into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape data for LSTM input [samples, timesteps, features]
X_train_lstm = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# One-hot encode labels for LSTM
y_train_lstm = to_categorical(y_train, num_classes=len(np.unique(y_train)))
y_test_lstm = to_categorical(y_test, num_classes=len(np.unique(y_test)))

# Build the LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(64, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(len(np.unique(y_train)), activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=1500, batch_size=32, validation_data=(X_test_lstm, y_test_lstm))

# Evaluate the LSTM model
lstm_accuracy = lstm_model.evaluate(X_test_lstm, y_test_lstm, verbose=0)
print(f"LSTM Accuracy: {lstm_accuracy[1] * 100:.2f}%")

# Generate the classification report
y_pred = np.argmax(lstm_model.predict(X_test_lstm), axis=1)
y_true = np.argmax(y_test_lstm, axis=1)
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

# Save the trained LSTM model to a specified folder
save_dir = "C:/Users/Ble Salva/Documents/EA/Dissert(2024)/ai/gesture_model"

# Ensure the directory exists (creates if not)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Specify the model file path (you can name the model file as you want)
model_file_path = os.path.join(save_dir, "lstm_model_combined.h5")

# Save the trained model
lstm_model.save(model_file_path)
print(f"Model saved to {model_file_path}")

# Real-time prediction function for LSTM
def real_time_predict_lstm(new_data):
    # Reshape and preprocess new_data (normalize and reshape for LSTM input)
    new_data_scaled = scaler.transform(new_data)
    new_data_reshaped = np.reshape(new_data_scaled, (new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))
    return np.argmax(lstm_model.predict(new_data_reshaped), axis=1)


In [ ]:
import time
import socket  # TCP client library
import requests
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from io import StringIO

# TCP server details
TCP_SERVER_HOST = "127.0.0.1"  # Unity server IP
TCP_SERVER_PORT = 5005         # Unity server port

# Load the trained model
model_path = "C:/Users/Ble Salva/Documents/EA/Dissert(2024)/ai/gesture_model/lstm_model_combined.h5"
lstm_model = load_model(model_path)

# Initialize the StandardScaler for normalization 
scaler = StandardScaler()

# ThingSpeak Channel and API key details
channel_id = '2667543'
read_api_key = 'HTJSEZCKZVSGUEGY'
results = 2

# Define the gesture mapping
label_mapping = {0: 'left', 1: 'right', 2: 'up', 3: 'down', 4: 'rest'}

def fetch_latest_data():
    """Fetch the latest sensor data from ThingSpeak."""
    url = f"https://api.thingspeak.com/channels/{channel_id}/feeds.csv?api_key={read_api_key}&results={results}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            csv_data = StringIO(response.text)
            df = pd.read_csv(csv_data)
            data = df[['field1', 'field2', 'field3', 'field4', 'field5', 'field6']]
            data.columns = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
            return data
        else:
            print(f"Error fetching data: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

def preprocess_data(data):
    """Preprocess data by normalizing and reshaping it for the LSTM model."""
    for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    data.dropna(inplace=True)
    data_scaled = scaler.fit_transform(data)
    data_reshaped = np.reshape(data_scaled, (data_scaled.shape[0], 1, data_scaled.shape[1]))
    return data_reshaped

def real_time_predict_lstm():
    """Fetch the latest data, preprocess it, and make a prediction."""
    data = fetch_latest_data()
    if data is not None and not data.empty:
        processed_data = preprocess_data(data)
        prediction = lstm_model.predict(processed_data)
        predicted_label = np.argmax(prediction, axis=1)[0]
        predicted_gesture = label_mapping.get(predicted_label, 'Unknown')
        print(f"Predicted Gesture: {predicted_gesture}")
        return predicted_gesture
    else:
        print("No valid data received.")
        return None

def send_prediction(tcp_socket, predicted_gesture):
    """Send the predicted gesture to Unity server via TCP."""
    try:
        tcp_socket.sendall(predicted_gesture.encode())
        print(f"Sent prediction: {predicted_gesture}")
    except Exception as e:
        print(f"Error sending prediction: {e}")

def main():
    """Main function for TCP connection and real-time predictions."""
    # Establishing a TCP connection
    tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    tcp_socket.connect((TCP_SERVER_HOST, TCP_SERVER_PORT))
    print("Connected to Unity server")

    try:
        while True:
            predicted_gesture = real_time_predict_lstm()
            if predicted_gesture is not None:
                send_prediction(tcp_socket, predicted_gesture)
            time.sleep(0.5)
    except Exception as e:
        print(f"Error: {e}")
    finally:
        tcp_socket.close()
        print("Connection closed")

# Run the script
if __name__ == "__main__":
    main()


Connected to Unity server
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
Predicted Gesture: left
Sent prediction: left
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Predicted Gesture: left
Sent prediction: left
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted Gesture: left
Sent prediction: left
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted Gesture: right
Sent prediction: right
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Predicted Gesture: right
Sent prediction: right
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Gesture: left
Sent prediction: left
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted Gesture: left
Sent prediction: left
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted Gesture: down
Sent prediction: down
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted Gesture: right
Sent prediction: right
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted Gesture: right
Sent prediction: right
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Predicted Gesture: rest
Sent prediction: rest
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Pre

In [ ]:
import time
import websocket  # WebSocket client library
import requests
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from io import StringIO

# WebSocket server details
WS_SERVER_URL = "ws://127.0.0.1:5000"  # Unreal Engine WebSocket server URL

# Load the trained model
model_path = "C:/Users/Ble Salva/Documents/EA/Dissert(2024)/ai/gesture_model/lstm_model_combined.h5"
lstm_model = load_model(model_path)

# Initialize the StandardScaler for normalization (match it to the one used during training)
scaler = StandardScaler()

# ThingSpeak Channel and API key details
channel_id = '2667543'
read_api_key = 'HTJSEZCKZVSGUEGY'
results = 2

# Define the gesture mapping
label_mapping = {0: 'left', 1: 'right', 2: 'up', 3: 'down', 4: 'rest'}

def fetch_latest_data():
    """Fetch the latest sensor data from ThingSpeak."""
    url = f"https://api.thingspeak.com/channels/{channel_id}/feeds.csv?api_key={read_api_key}&results={results}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            csv_data = StringIO(response.text)
            df = pd.read_csv(csv_data)
            data = df[['field1', 'field2', 'field3', 'field4', 'field5', 'field6']]
            data.columns = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
            return data
        else:
            print(f"Error fetching data: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

def preprocess_data(data):
    """Preprocess data by normalizing and reshaping it for the LSTM model."""
    for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    data.dropna(inplace=True)
    data_scaled = scaler.fit_transform(data)
    data_reshaped = np.reshape(data_scaled, (data_scaled.shape[0], 1, data_scaled.shape[1]))
    return data_reshaped

def real_time_predict_lstm():
    """Fetch the latest data, preprocess it, and make a prediction."""
    data = fetch_latest_data()
    if data is not None and not data.empty:
        processed_data = preprocess_data(data)
        prediction = lstm_model.predict(processed_data)
        predicted_label = np.argmax(prediction, axis=1)[0]
        predicted_gesture = label_mapping.get(predicted_label, 'Unknown')
        print(f"Predicted Gesture: {predicted_gesture}")
        return predicted_gesture
    else:
        print("No valid data received.")
        return None

def send_prediction(ws, predicted_gesture):
    """Send the predicted gesture to Unreal Engine via WebSocket."""
    try:
        ws.send(predicted_gesture)
        print(f"Sent prediction: {predicted_gesture}")
    except Exception as e:
        print(f"Error sending prediction: {e}")

def main():
    """Main function for WebSocket connection and real-time predictions."""
    ws = websocket.WebSocket()
    ws.connect(WS_SERVER_URL)

    while True:
        try:
            predicted_gesture = real_time_predict_lstm()
            if predicted_gesture is not None:
                send_prediction(ws, predicted_gesture)
            time.sleep(0.5)
        except Exception as e:
            print(f"Error: {e}")
            ws.close()
            break

# Run the script
if __name__ == "__main__":
    main()


In [ ]:
import requests
import time
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from io import StringIO

# Step 2: Load the trained LSTM model 
model_path = "C:/Users/Ble Salva/Documents/EA/Dissert(2024)/ai/gesture_model/lstm_model_combined.h5"
lstm_model = load_model(model_path)

# Initialize the StandardScaler for normalization (ensure it matches the one used during training)
scaler = StandardScaler()

# ThingSpeak Channel and API key details
channel_id = '2667543'  # Replace with your channel ID
read_api_key = 'HTJSEZCKZVSGUEGY'  # Replace with your ThingSpeak read API key
results = 10  # Fetch one data point for real-time prediction

# Define the gesture mapping (adjust based on your model's label encoding)
label_mapping = {0: 'left', 1: 'right', 2: 'up', 3: 'down', 4: 'rest'}

def fetch_latest_data():
    """Fetch the latest sensor data from ThingSpeak."""
    url = f"https://api.thingspeak.com/channels/{channel_id}/feeds.csv?api_key={read_api_key}&results={results}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            # Load the CSV data into a pandas DataFrame
            csv_data = StringIO(response.text)
            df = pd.read_csv(csv_data)

            # Map the columns accordingly
            # Ensuring your columns are ['created_at', 'entry_id', 'field1', 'field2', ..., 'field7']
            data = df[['field1', 'field2', 'field3', 'field4', 'field5', 'field6']]
            data.columns = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']

            return data
        else:
            print(f"Error fetching data: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Exception occurred: {e}")
        return None

def preprocess_data(data):
    """Preprocess data by normalizing and reshaping it for the LSTM model."""
    # Convert the data to numeric (just in case)
    for col in ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    # Drop rows with NaN values
    data.dropna(inplace=True)

    # Normalize the data using the scaler (ensure this scaler matches the one used during training)
    data_scaled = scaler.fit_transform(data)

    # Reshape the data for LSTM input (1 sample, 1 timestep, 6 features)
    data_reshaped = np.reshape(data_scaled, (data_scaled.shape[0], 1, data_scaled.shape[1]))

    return data_reshaped

def real_time_predict_lstm():
    """Fetch the latest data from ThingSpeak, preprocess it, and make a prediction using the LSTM model."""
    # Fetch the latest data from ThingSpeak
    data = fetch_latest_data()

    if data is not None and not data.empty:
        # Preprocess the data
        processed_data = preprocess_data(data)

        # Make the prediction
        prediction = lstm_model.predict(processed_data)

        # Convert prediction to label (argmax to get the predicted class)
        predicted_label = np.argmax(prediction, axis=1)[0]

        # Map the predicted label to the corresponding gesture
        predicted_gesture = label_mapping.get(predicted_label, 'Unknown')

        print(f"Predicted Gesture: {predicted_gesture}")
        return predicted_gesture
    else:
        print("No valid data received.")
        return None

# Run real-time prediction loop (or call real_time_predict_lstm() when needed)
if __name__ == "__main__":
    while True:
        # Fetch and predict every few seconds (simulate real-time)
        real_time_predict_lstm()
        time.sleep(0.5)  # Adjust the sleep interval as needed

In [ ]:
import socket
import time
import pickle  # To send serialized model predictions

# Create a TCP/UDP client
HOST = 'localhost'  # IP of the UE5 server
PORT = 5000         # Same port as the server will listen to

def predict_direction(input_data, model):
    """
    A placeholder function for predicting direction using an AI model.
    Replace this with your actual AI model prediction code.
    """
    # Use the AI model to make a prediction: e.g., 'left', 'right', 'up', 'down', 'rest'
    prediction = model.predict(input_data)
    return prediction

def send_prediction(prediction):
    # Create a TCP socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        # Send the prediction serialized as bytes
        s.sendall(pickle.dumps(prediction))

def main(model):
    while True:
        # Simulate AI input data for the model (replace with actual data)
        input_data = get_input_data()

        # Get prediction from the AI model
        direction = predict_direction(input_data, model)

        # Send the predicted direction to Unreal Engine
        send_prediction(direction)

        # Wait before sending the next prediction (adjust to your needs)
        time.sleep(0.1)

# Run the script
if __name__ == "__main__":
    # Load your trained AI model here
    model = load_your_model()  # Placeholder function for model loading
    main(model)
